## **Turi Object Detection**
Welcome to the object detection model quickstart on Skafos! The purpose of this notebook is to get you going end-to-end. Below we will do the following:

1. Load images of cars, bikes and people and their corresponding [bounding boxes](https://computersciencewiki.org/index.php/Bounding_boxes).
2. Build an object detection model for detecting cars, bikes and people in images.
3. Convert the model to CoreML format and save it to the Saves framework.

The example is based on Turi Create's Object Detection model.

---

Execute each cell one-by-one, by selecting the cell and do one of the following:

- Clicking the "play" button at the top of this frame.
- Typing 'Control + Enter' or 'Shift + Enter'.

In [ ]:
# Import necessary librariees
import tarfile
import urllib.request

from skafossdk import *
import utilities.save_models as sm
from utilities.load_turicreate import *

tc = install_turicreate(timeout=500, retries=2) # check to make sure turicreate is installed

import coremltools


In [ ]:
ska = Skafos() ## initialize Skafos

### 1. **Load the data**
The data for this example are 1096 images, which breaks down to 420 cars and 365 bikes and 311 people. The data set can be found [here](https://lear.inrialpes.fr/people/marszalek/data/ig02/).  To see how this data was created from the raw data set, check out `advanced_usage/load_object_data.ipynb`. The data is randomly split into train and test sets, where 80% of the data is used for training, and 20% is used for model evaluation.

In [ ]:
# Specify the data set download url
data_url = "https://s3.amazonaws.com/skafos.example.data/ObjectDetection/ig02.sframe.tar.gz"
data_path = "ig02.sframe.tar.gz"

## Pull the compressed data and extract it
retrieve = urllib.request.urlretrieve(data_url, data_path)
tar = tarfile.open(data_path)
tar.extractall()
tar.close()

In [ ]:
## Load the data
data =  tc.SFrame('ig02.sframe')

## Make a train-test split
train_data, test_data = data.random_split(0.8)

### 2. **Build the model**
We pass the data to the `tc.object_detector.create` function and specify a few arguments needed to properly run the model. To understand more about this specific function, check out the [Turi Create Documentation](https://apple.github.io/turicreate/docs/userguide/object_detection/).

**_Note: we highly recommend running this on a GPU. We encourage you to do this once you've changed the data to reflect your use case. As benchmarks, we've found this takes about 2 hours on a GPU, about 1.5 days on Skafos with 6 CPU's and 10G of memory. For this reason, we've commented out the model running and model saving below._**

In [ ]:
## Create a model
#model = tc.object_detector.create(dataset=train_data,
#                                  max_iterations=0, # if 0, Turi Create determines # of iterations
#)

### 3. **Save the model**
Once your model has been created, it must be saved to the Skafos framework via the code below. This will trigger a push to your mobile app. Note that you will need to uncomment the lines below to do this.

In [ ]:
## Export the model to CoreML

# Specify the CoreML model name
#coreml_model_name = 'ObjectDetection.mlmodel'
# Specify the name of the compressed model we save to Skafos
#compressed_model_name = coreml_model_name + ".gz" 

# Export the trained model to CoreML format
#res = model.export_coreml(coreml_model_name) 

## Use coremltools to convert model weights to half-precision.
## This may be necessary if you have memory concerns within your app
#model_spec = coremltools.utils.load_spec(coreml_model_name)
#model_fp16_spec = coremltools.utils.convert_neural_network_spec_weights_to_fp16(model_spec)
#coremltools.utils.save_spec(model_fp16_spec, coreml_model_name)

# Compress the model
#compressed_model = sm.compress_model(coreml_model_name)

# Save to Skafos
#sm.skafos_save_model(
#    skafos=ska, 
#    model_name=compressed_model_name,
#    compressed_model=compressed_model,
#    permissions='public'
# )